In [2]:
import pandas as pd

# prenoms inventés
mean_diff = pd.read_csv("data/clean/influenced_names_means_diff.csv")
influenced_meandiff = mean_diff[mean_diff["Influence"] > 0]
# removing common identification mistakes such as "the", "a" or "Mr"
influenced_meandiff = influenced_meandiff[~influenced_meandiff["Character Name"].isin(["the", "a", "Mr"])]
print("Number of influenced names with mean diff: ", len(influenced_meandiff))
influenced_meandiff.head()



Number of influenced names with mean diff:  1585


,Wikipedia ID,Movie Name,Year,Character Name,Count,Full name,Normalized_name,Influence
0,31186339,the hunger games,2012,Katniss,4,Katniss,KATNISS,inf
1,22144721,iron man 2,2010,Stark,3,Howard Stark,STARK,inf
2,146947,spider-man,2002,Osborn,6,Harry Osborn,OSBORN,inf
4,443972,hook,1991,Banning,16,Peter Banning,BANNING,inf
5,537416,ace ventura: when nature calls,1995,Abbot,2,Grand Abbot,ABBOT,inf


In [3]:
# tous les prénoms influencés
prophet = pd.read_csv("data/clean/influenced_names_prophet.csv")
influenced_prophet = prophet[prophet["Influenced"] > 0]
# removing common identification mistakes such as "the", "a" or "Mr"
influenced_prophet = influenced_prophet[~influenced_prophet["Character Name"].isin(["the", "a", "Mr"])]
print("Number of influenced names with mean diff: ", len(influenced_prophet))
influenced_prophet.head()

Number of influenced names with mean diff:  432


,Wikipedia ID,Movie Name,Year,Character Name,Count,Full name,Normalized_name,Mean Difference,Influenced
0,451866,mission: impossible ii,2000,Ethan,18,Ethan Hunt,ETHAN,15725.466667,1
1,633411,the avengers,1998,Emma,15,Emma Peel,EMMA,14985.966667,1
2,3727473,man on fire,1987,Samantha,4,"Samantha ""Sam"" Balletto",SAMANTHA,14453.500000,1
3,347000,suspiria,1977,Sarah,15,Sarah,SARAH,14372.466667,1
5,320401,barton fink,1991,Taylor,3,Audrey Taylor,TAYLOR,13892.100000,1


In [4]:
#remove every non numeric value in mean diff
mean_diff = mean_diff[mean_diff["Influence"].apply(lambda x: str(x).replace(".", "").isdigit())]
threshold = mean_diff['Influence'].quantile(0.75)

significant_names = mean_diff[mean_diff['Influence'] > threshold]
print("Number of significant names:", len(significant_names))
significant_names.head()

Number of significant names: 371


,Wikipedia ID,Movie Name,Year,Character Name,Count,Full name,Normalized_name,Influence
151,451866,mission: impossible ii,2000,Ethan,18,Ethan Hunt,ETHAN,15725.466667
152,633411,the avengers,1998,Emma,15,Emma Peel,EMMA,14985.966667
153,3727473,man on fire,1987,Samantha,4,"Samantha ""Sam"" Balletto",SAMANTHA,14453.500000
154,347000,suspiria,1977,Sarah,15,Sarah,SARAH,14372.466667
155,483274,point break,1991,Tyler,3,Tyler Endicott,TYLER,14176.666667


In [7]:
cmu_imdb_merged = pd.read_csv("data/clean/cmu_imdb_merged.csv")
cmu_imdb_merged.head(1)

,Wikipedia_movie_ID,Movie_name,Release_date,Revenue,Runtime,Languages,Countries,Genres,weightedAverageRating,totalVotes,is_blockbuster
0,29988427.0,!women art revolution,2010-01-01,NaN,0 days 01:23:00,English,"United States of America, Canada","LGBT, History, Documentary",6.9,262.0,False


In [54]:
# Perform a merge based on 'Wikipedia ID' (from influenced_prophet) and 'Wikipedia_movie_ID' (from cmu_imdb_merged)
merged_df = influenced_prophet.merge(
    cmu_imdb_merged[['Wikipedia_movie_ID', 'Genres']],  # Keep only relevant columns
    left_on="Wikipedia ID", 
    right_on="Wikipedia_movie_ID", 
    how="left"
)
# Drop redundant column after the merge
#merged_df.drop(columns=['Wikipedia_movie_ID'], inplace=True)

# Display the result
print("Merged Dataset:")
display(merged_df.head(1))

# Save the new dataset with Genres
#merged_df.to_csv("data/clean/influenced_prophet_with_genres.csv", index=False)
#print("Dataset saved successfully with Genres added!")


Merged Dataset:


,Wikipedia ID,Movie Name,Year,Character Name,Count,Full name,Normalized_name,Mean Difference,Influenced,Wikipedia_movie_ID,Genres
0,451866,mission: impossible ii,2000,Ethan,18,Ethan Hunt,ETHAN,15725.466667,1,451866.0,"Thriller, Action Thrillers, Action/Adventure, ..."


In [95]:
merged_df['Genres'] = merged_df['Genres'].str.split(', ')
merged_df.head(1)


,Wikipedia ID,Movie Name,Year,Character Name,Count,Full name,Normalized_name,Mean Difference,Influenced,Wikipedia_movie_ID,Genres
0,451866,mission: impossible ii,2000,Ethan,18,Ethan Hunt,ETHAN,15725.466667,1,451866.0,NaN


In [43]:
exploded_df = merged_df.explode('Genres')


In [94]:
genre_counts = exploded_df['Genres'].value_counts()


In [45]:
genre_mean_influence = exploded_df.groupby('Genres')['Mean Difference'].mean().sort_values(ascending=False)


In [46]:
# 5. Afficher les résultats
print("Nombre de prénoms influencés par genre :")
display(genre_counts.head(10))  # Top 10 genres

print("\nInfluence moyenne par genre :")
display(genre_mean_influence.head(10))  # Top 10 genres par influence moyenne



Nombre de prénoms influencés par genre :


Genres
Drama               210
Thriller            159
Action              159
Action/Adventure    128
Comedy              121
Crime Fiction       100
Romance Film         93
Adventure            88
Science Fiction      86
Horror               58
Name: count, dtype: int64


Influence moyenne par genre :


Genres
Glamorized Spy Film          9425.227778
Gothic Film                  8296.677778
Spy                          6740.085417
Costume Horror               5517.866667
Auto racing                  4056.733333
Heaven-Can-Wait Fantasies    2943.966667
Blaxploitation               2534.433333
Buddy cop                    2504.611111
Action Thrillers             2435.469608
Supernatural                 2401.844444
Name: Mean Difference, dtype: float64

## Problème de la facon de faire précéddente : count et pas valeur de la mean

In [ ]:
import pandas as pd


# Step 1: Explode 'Genres' to separate multiple genres per row
exploded_df = sorted_df.copy()
exploded_df['Genres'] = exploded_df['Genres'].str.split(', ')
exploded_df = exploded_df.explode('Genres')

# Step 2: Group by Genres and calculate the total Mean Difference
genre_influence = (
    exploded_df.groupby('Genres')['Mean Difference']
    .sum()
    .reset_index()
    .sort_values(by='Mean Difference', ascending=False)
)

# Step 3: Display the top 10 most influential genres
print(genre_influence.head(10))

# Optional: Save results to CSV
genre_influence.to_csv("top_influential_genres.csv", index=False)

# Visualization (optional): Plot a bar chart for the top 10 genres
import plotly.express as px

fig = px.bar(
    genre_influence.head(10),
    x='Genres',
    y='Mean Difference',
    title='Top 10 Most Influential Movie Genres on Names',
    labels={'Mean Difference': 'Total Influence Score'},
    template='plotly_white'
)
fig.show()


### new trial

In [83]:
import pandas as pd
import plotly.graph_objects as go

# Étape 1: Préparer les données
merged_df = pd.read_csv("data/clean/influenced_prophet_with_genres.csv")
merged_df['Genres'] = merged_df['Genres'].str.split(', ')
exploded_df = merged_df.explode('Genres')  # Séparer les genres

# Calculer l'influence totale par genre
top_25_genres = (
    exploded_df.groupby('Genres')['Mean Difference']
    .sum()
    .reset_index()
    .sort_values(by='Mean Difference', ascending=False)
    .head(25)['Genres']  # Garde les 25 genres les plus influents
)

# Filtrer les données pour ne garder que ces 25 genres
filtered_df = exploded_df[exploded_df['Genres'].isin(top_25_genres)]

# Calculer le Top 3 des prénoms influencés pour chaque genre
top_names_by_genre = (
    filtered_df.groupby(['Genres', 'Normalized_name'])['Mean Difference']
    .sum()
    .reset_index()
    .sort_values(['Genres', 'Mean Difference'], ascending=[True, False])
    .groupby('Genres')
    .head(3)  # Prendre le top 3 par genre
)

# Liste unique des 25 genres
unique_genres = top_names_by_genre['Genres'].unique()

# Étape 2: Créer le graphique initial avec le premier genre
initial_genre = unique_genres[0]
initial_data = top_names_by_genre[top_names_by_genre['Genres'] == initial_genre]

# Création de la figure
fig = go.Figure()

# Ajouter les barres pour le graphique initial
fig.add_trace(go.Bar(
    x=initial_data['Normalized_name'],
    y=initial_data['Mean Difference'],
    name=initial_genre
))

# Étape 3: Configurer les étapes du slider
steps = []
for genre in unique_genres:
    filtered_data = top_names_by_genre[top_names_by_genre['Genres'] == genre]
    step = dict(
        method="update",
        args=[{"x": [filtered_data['Normalized_name']],
               "y": [filtered_data['Mean Difference']],
               "type": "bar"},
              {"title": f"Top 3 prénoms influencés par le genre '{genre}'"}],
        label=genre
    )
    steps.append(step)

# Étape 4: Configuration du slider
sliders = [dict(
    active=0,
    currentvalue={"prefix": "Genre: "},
    pad={"t": 50},
    steps=steps
)]

# Mettre à jour le layout
fig.update_layout(
    title=f"Top 3 influences names by movies, per genre'{initial_genre}'",
    xaxis_title="Names",
    yaxis_title="Mean of influence",
    sliders=sliders,
    template="plotly_white"
)

# Étape 5: Afficher le graphique
fig.show()


In [82]:
fig.write_html("docs/_includes/top3_influenced_names_by_genres.html")

In [85]:
!pip install dash plotly pandas



In [87]:
import pandas as pd
import dash
from dash import dcc, html, Input, Output
import plotly.express as px
import plotly.graph_objects as go

# Charger les données
merged_df = pd.read_csv("data/clean/influenced_prophet_with_genres.csv")
merged_df['Genres'] = merged_df['Genres'].str.split(', ')
exploded_df = merged_df.explode('Genres')

# Top 25 genres les plus influents
top_25_genres = (
    exploded_df.groupby('Genres')['Mean Difference']
    .sum()
    .reset_index()
    .sort_values(by='Mean Difference', ascending=False)
    .head(25)['Genres']
)

filtered_df = exploded_df[exploded_df['Genres'].isin(top_25_genres)]

# Obtenir le Top 3 des prénoms influencés par genre
top_names_by_genre = (
    filtered_df.groupby(['Genres', 'Normalized_name'])['Mean Difference']
    .sum()
    .reset_index()
    .sort_values(['Genres', 'Mean Difference'], ascending=[True, False])
    .groupby('Genres')
    .head(3)
)

# Liste unique des genres
unique_genres = top_names_by_genre['Genres'].unique()



In [93]:
import pandas as pd
import dash
from dash import dcc, html, Input, Output
import plotly.express as px
import plotly.graph_objects as go

# Charger les données
merged_df = pd.read_csv("data/clean/influenced_prophet_with_genres.csv")
merged_df['Genres'] = merged_df['Genres'].str.split(', ')
exploded_df = merged_df.explode('Genres')

# Top 25 genres les plus influents
top_25_genres = (
    exploded_df.groupby('Genres')['Mean Difference']
    .sum()
    .reset_index()
    .sort_values(by='Mean Difference', ascending=False)
    .head(25)['Genres']
)

filtered_df = exploded_df[exploded_df['Genres'].isin(top_25_genres)]

# Obtenir le Top 3 des prénoms influencés par genre
top_names_by_genre = (
    filtered_df.groupby(['Genres', 'Normalized_name'])['Mean Difference']
    .sum()
    .reset_index()
    .sort_values(['Genres', 'Mean Difference'], ascending=[True, False])
    .groupby('Genres')
    .head(3)
)

# Liste unique des genres
unique_genres = top_names_by_genre['Genres'].unique()

# Fonction pour générer une grille hexagonale compacte
def generate_hex_coords_compact(n, cols=6):
    """Générer des coordonnées x et y pour une grille hexagonale compacte."""
    coords = []
    for i in range(n):
        row = i // cols
        col = i % cols
        x = col * 1.5  # Espacement horizontal
        y = row * 1.3  # Espacement vertical
        if row % 2 == 1:  # Décalage horizontal pour les lignes impaires
            x += 0.75
        coords.append((x, -y))
    return coords

# Générer les coordonnées pour les genres
coords = generate_hex_coords_compact(len(unique_genres), cols=6)
x_coords, y_coords = zip(*coords)

# Initialiser l'application Dash
app = dash.Dash(__name__)

# Layout de l'application
app.layout = html.Div([
    html.H1("Select a movie genre:", style={"text-align": "center"}),

    # Grille hexagonale interactive
    dcc.Graph(
        id='hex-graph',
        figure=px.scatter(
            x=x_coords, y=y_coords,
            text=unique_genres,
            template='simple_white'
        ).update_traces(
            marker=dict(symbol="hexagon", size=70, color='blue'),
            textposition="middle center",
            hoverinfo="text"
        ).update_layout(
            xaxis=dict(visible=False),
            yaxis=dict(visible=False),
            height=600,
            margin=dict(l=0, r=0, t=0, b=0)
        )
    ),

    # Barplot dynamique pour les prénoms
    dcc.Graph(id='bar-plot')
])

# Callback pour mettre à jour le barplot
@app.callback(
    Output('bar-plot', 'figure'),
    Input('hex-graph', 'hoverData')  # Interaction via hover
)
def update_barplot(hoverData):
    # Récupérer le genre sélectionné (hover)
    if hoverData is not None:
        selected_genre = hoverData['points'][0]['text']
    else:
        selected_genre = unique_genres[0]  # Par défaut

    # Filtrer le top 3 des prénoms pour le genre sélectionné
    filtered_names = top_names_by_genre[top_names_by_genre['Genres'] == selected_genre]

    # Créer un barplot
    fig = go.Figure(data=[
        go.Bar(
            x=filtered_names['Normalized_name'],
            y=filtered_names['Mean Difference'],
            marker_color='lightskyblue'
        )
    ])
    fig.update_layout(
        title=f"Top 3 prénoms influencés par le genre '{selected_genre}'",
        xaxis_title="Prénoms",
        yaxis_title="Mean of Influence",
        template="simple_white"
    )
    return fig

# Exécuter l'application
if __name__ == '__main__':
    app.run_server(debug=True)
